In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
from statannot import add_stat_annotation
sys.path.append("../")

from utils.io import read_parameters

In [ ]:
plt.rcParams.update({'font.size': 18})

parameter_file  = "../local/parameters.yml"
parameters = read_parameters(parameter_file)   

base_folder = parameters["base_folder"]
tracking_data_files = parameters["tracking_data"]
data_exclude = parameters["data_exclude"]
output_folder = parameters["output_folder"]

single_time_point_evaluation = parameters["single_time_point_evaluation"]

interval = parameters["time_lag"]
hue_order = ["siScr", "siCdc42", "siRac1"]

decimal_places = parameters["decimal_places"]

In [ ]:
parameters

# load data

In [ ]:
tracking_data_df = pd.DataFrame() 
column_dtypes = {'TRACK_ID': 'int16', 
                 'FRAME': 'int16', 
                             'POSITION_X' : 'float16',
                             'POSITION_Y' : 'float16',
                             'POSITION_T' : 'float16'}
print(list(column_dtypes))

for condition in tracking_data_files:
    for filename in tracking_data_files[condition]:
        
        if filename in data_exclude:
            print("Excluding file: ", filename)
            print("##################")
            continue
        
        print("Processing file: ", filename)
        data = pd.read_csv(base_folder + "/"+ filename, low_memory=False).drop([0,1,2])

        data_ = data[list(column_dtypes)]

        data_.insert(0, "filename", filename)
        data_.insert(0, "condition", condition)
        
        data_ = data_.astype(column_dtypes)
        data_ = data_.sort_values(by= "FRAME")
        
        for track_id in data_["TRACK_ID"].unique():
            single_track_df = data_[data_["TRACK_ID"]==track_id]
            track_length = len(single_track_df.index) 
            start_frame = single_track_df["FRAME"].min()
            end_frame = single_track_df["FRAME"].max()
            ### uncomment to check for gaps
            #if track_length < end_frame - start_frame + 1:
            #    print("Track: ", track_id, "with length ", track_length, " has a gap")
            #    print(np.array(single_track_df["FRAME"]))
            #else: 
            #    print("Track: ", track_id, "with length ", track_length," has no gap")
            #    print(np.array(single_track_df["FRAME"]))
            ###
            start_x = np.array(single_track_df["POSITION_X"])[0]
            start_y = np.array(single_track_df["POSITION_Y"])[0]
            data_.loc[data_.TRACK_ID == track_id, "START_X"] = start_x
            data_.loc[data_.TRACK_ID == track_id, "START_Y"] = start_y
            
            if track_length < parameters["min_track_length"]:
                data_ = data_[data_["TRACK_ID"] != track_id ]
                
        print("##################")
        if len(tracking_data_df.index) > 10:
            #tracking_data_df = tracking_data_df.append(data_)
            tracking_data_df = pd.concat([tracking_data_df, data_], ignore_index = True)
        else:
            tracking_data_df = data_.copy()




In [ ]:
### for trajectory plots            
tracking_data_df["ORIGIN_X"] = tracking_data_df["POSITION_X"] - tracking_data_df["START_X"] 
tracking_data_df["ORIGIN_Y"] = tracking_data_df["POSITION_Y"] - tracking_data_df["START_Y"] 
tracking_data_df["ORIGIN_L"] = np.sqrt(tracking_data_df["ORIGIN_X"]**2 + tracking_data_df["ORIGIN_Y"]**2)

In [ ]:
display(tracking_data_df)

In [ ]:
tracking_data_df.to_csv(output_folder + "tracking_data.csv")

In [ ]:
# plot info for a single track from a single file
test = tracking_data_df[tracking_data_df["TRACK_ID"] ==0].sort_values(by= "FRAME")
test = test[test["filename"] == "/siScr/SUM_230420_siScr_20dyn_TrackMate.csv"]
display(test)
len(test.index)

# Quality check: plot abundance and length of trajectories

In [ ]:
for filename in tracking_data_df["filename"].unique():
    tracking_data_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    fig, ax = plt.subplots(figsize=(20,10))
    sns.scatterplot(data = tracking_data_df_, x = "FRAME", y = "TRACK_ID")
    ax.set_title(filename)

In [ ]:
for filename in tracking_data_df["filename"].unique():
    tracking_data_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    fig, ax = plt.subplots(figsize=(20,10))
    tracking_data_df_[["FRAME","TRACK_ID"]].groupby("FRAME").count().plot(ax =ax)
    ax.set_title(filename)

# Split data into phase 1 and phase 2

In [ ]:
phase_1_data_df = tracking_data_df[tracking_data_df["FRAME"] <= parameters["end_phase_1"]]
phase_2_data_df = tracking_data_df[tracking_data_df["FRAME"] >= parameters["start_phase_2"]]

In [ ]:
test_df = phase_1_data_df[phase_1_data_df["filename"] == "/siRac1/SUM_230427_siRac1_TrackMate.csv"]
print(test_df["TRACK_ID"].unique())
single_track_test_df = test_df[test_df["TRACK_ID"] == 379]
single_track_test_df 

# Plot trajectories

In [ ]:
subsampling_n = 10

# phase 1 trajectories split by filename (all trajctories)

In [ ]:
for filename in phase_1_data_df["filename"].unique():
    tracking_data_df_ = phase_1_data_df[phase_1_data_df["filename"] == filename]

    fig, ax = plt.subplots(figsize=(9,9))
    for track_id in tracking_data_df_["TRACK_ID"].unique():
        single_track_df = tracking_data_df_[tracking_data_df_["TRACK_ID"]==track_id]
        if len(single_track_df.index) < parameters["end_phase_1"]:
            continue

        end_x = np.array(single_track_df["ORIGIN_X"])[-1]
        end_y = np.array(single_track_df["ORIGIN_Y"])[-1]
        
        if end_x < 0.0:
            ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "#377eb8")
            ax.plot([end_x],[end_y], color = "black", marker = "o")  
        else:
            ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "#ff7f00")
            ax.plot([end_x],[end_y], color = "black", marker = "o") 
            
        #ax.plot([end_x],[end_y], color = "red", marker = "o")
        
    ax.set_xlim(-50,50)
    ax.set_ylim(-50,50)
    ax.axhline(0, color="red", linestyle = "--")
    ax.axvline(0, color="red", linestyle = "--")
    ax.set_title(filename)
    plt.savefig(output_folder + "EC_trajectories_%s.png" % str(filename).split("/")[2])
    plt.savefig(output_folder + "EC_trajectories_%s.pdf" % str(filename).split("/")[2])

# phase 1 trajectories split by filename (every n-th trajectories)

In [ ]:
for filename in phase_1_data_df["filename"].unique():
    tracking_data_df_ = phase_1_data_df[phase_1_data_df["filename"] == filename]

    fig, ax = plt.subplots(figsize=(10,10))
    counter = 0
    for track_id in tracking_data_df_["TRACK_ID"].unique():
        single_track_df = tracking_data_df_[tracking_data_df_["TRACK_ID"]==track_id]
        if len(single_track_df.index) < parameters["end_phase_1"]:
            continue
        if counter % subsampling_n == 0: 
            end_x = np.array(single_track_df["ORIGIN_X"])[-1]
            end_y = np.array(single_track_df["ORIGIN_Y"])[-1]
        
            if end_x < 0.0:
                ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "black")
                ax.plot([end_x],[end_y], color = "black", marker = "o")  
            else:
                ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "orange")
                ax.plot([end_x],[end_y], color = "orange", marker = "o") 
        counter += 1
        
    ax.set_xlim(-40,40)
    ax.set_ylim(-40,40)
    ax.set_title(filename)
    ax.axhline(0, color="red", linestyle = "--")
    ax.axvline(0, color="red", linestyle = "--")
    plt.savefig(output_folder + "EC_trajectories_subsample_%s_%s.png" % (subsampling_n,str(filename).split("/")[2]))
    plt.savefig(output_folder + "EC_trajectories_subsample_%s_%s.pdf" % (subsampling_n,str(filename).split("/")[2]))

# phase 1 trajectories split by condition (all trajctories)

In [ ]:
for condition in phase_1_data_df["condition"].unique():
    condition_data_df = phase_1_data_df[phase_1_data_df["condition"] == condition]

    fig, ax = plt.subplots(figsize=(10,10))  
    
    for filename in condition_data_df["filename"].unique():

        tracking_data_df_ = condition_data_df[condition_data_df["filename"] == filename ]
        for track_id in tracking_data_df_["TRACK_ID"].unique():
            single_track_df = tracking_data_df_[tracking_data_df_["TRACK_ID"]==track_id]
            if len(single_track_df.index) < parameters["end_phase_1"]:
                continue

            end_x = np.array(single_track_df["ORIGIN_X"])[-1]
            end_y = np.array(single_track_df["ORIGIN_Y"])[-1]

            if end_x < 0.0:
                ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "black")
                ax.plot([end_x],[end_y], color = "black", marker = "o")  
            else:
                ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "orange")
                ax.plot([end_x],[end_y], color = "orange", marker = "o") 
            
        
    ax.set_xlim(-40,40)
    ax.set_ylim(-40,40)
    ax.axhline(0, color="red", linestyle = "--")
    ax.axvline(0, color="red", linestyle = "--")
    ax.set_title(condition)
    ax.set_xlabel("$\Delta$x in $\mu$m")
    ax.set_ylabel("$\Delta$y in $\mu$m")
    plt.savefig(output_folder + "EC_trajectories_%s.png" % condition)
    plt.savefig(output_folder + "EC_trajectories_%s.pdf" % condition)        
        

# phase 1 trajectories split by condition (every n-th trajectory)

In [ ]:
for condition in phase_1_data_df["condition"].unique():
    condition_data_df = phase_1_data_df[phase_1_data_df["condition"] == condition]

    fig, ax = plt.subplots(figsize=(10,10))
    
    for filename in condition_data_df["filename"].unique():
        counter = 0
        tracking_data_df_ = condition_data_df[condition_data_df["filename"] == filename ]
        for track_id in tracking_data_df_["TRACK_ID"].unique():
                   
            single_track_df = tracking_data_df_[tracking_data_df_["TRACK_ID"]==track_id]
            if len(single_track_df.index) < parameters["end_phase_1"]:
                continue
            if counter % subsampling_n == 0:  
                end_x = np.array(single_track_df["ORIGIN_X"])[-1]
                end_y = np.array(single_track_df["ORIGIN_Y"])[-1]

                if end_x < 0.0:
                    ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "black")
                    ax.plot([end_x],[end_y], color = "black", marker = "o")  
                else:
                    ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "orange")
                    ax.plot([end_x],[end_y], color = "orange", marker = "o") 
                 
            counter += 1
            
        
    ax.set_xlim(-40,40)
    ax.set_ylim(-40,40)
    ax.axhline(0, color="red", linestyle = "--")
    ax.axvline(0, color="red", linestyle = "--")
    ax.set_title(condition)
    ax.set_xlabel("$\Delta$x in $\mu$m")
    ax.set_ylabel("$\Delta$y in $\mu$m")
    plt.savefig(output_folder + "EC_trajectories_subsample_%s_%s.png" % (subsampling_n,condition))
    plt.savefig(output_folder + "EC_trajectories_subsample_%s_%s.pdf" % (subsampling_n,condition))     
        

# Compute migration velocities

In [ ]:
#migration_speed_df = pd.DataFrame(columns=tracking_data_df.columns)
migration_speed_df = pd.DataFrame()


interval = parameters["time_lag"]
for filename in tracking_data_df["filename"].unique():
    tracks_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    tracks_df = tracks_df_[["TRACK_ID", "POSITION_X", "POSITION_Y", "POSITION_T", "FRAME", "ORIGIN_X", "ORIGIN_Y"]]
    
    print(filename)
    status = 0
    tracks_num = len(tracks_df["TRACK_ID"].unique())
    
    for track_id in tracks_df["TRACK_ID"].unique():

        single_track_df = tracks_df[tracks_df ["TRACK_ID"]==track_id]
        single_track_df = single_track_df.sort_values(by="FRAME")
        dist = single_track_df.diff(interval).fillna(0.)
        dist["time_in_h"] = dist["FRAME"]*5.0/60.0

        single_track_df["step_size"] = np.round(np.sqrt(dist.POSITION_X**2 + dist.POSITION_Y**2),decimal_places)
        single_track_df["step_size_x"] = np.round(dist.POSITION_X,decimal_places)
        single_track_df["step_size_y"] =  np.round(dist.POSITION_Y,decimal_places)
        single_track_df["vel_mu_per_h"] = np.round(np.sqrt(dist.POSITION_X**2 + dist.POSITION_Y**2)/dist.time_in_h,decimal_places)
        single_track_df["vel_x_mu_per_h"] = np.round(dist.POSITION_X/dist.time_in_h,decimal_places)
        single_track_df["vel_y_mu_per_h"] =  np.round(dist.POSITION_Y/dist.time_in_h,decimal_places)
        
        single_track_df["phi"] =  np.round(np.arctan2(dist.POSITION_Y,-dist.POSITION_X)*180.0/np.pi,decimal_places)

        single_track_df["filename"] = filename
        single_track_df["condition"] = tracks_df_["condition"].iloc[0]
        
        single_track_df["time_in_h"] = np.round(single_track_df["FRAME"]*5.0/60.0,decimal_places)
        
        #print("Track ID %s" % track_id)
        #print(single_track_df.head())
        if len(migration_speed_df.index) > 1:
            #migration_speed_df = migration_speed_df.append(single_track_df, ignore_index=True)
            migration_speed_df = pd.concat( [migration_speed_df, single_track_df], ignore_index=True)
        else:
            migration_speed_df = single_track_df.copy()
        
        #migration_speed_df = migration_speed_df.append(single_track_df, ignore_index=True)
        
        status +=1 
        if status % 500 == 0:
            print("%s out of %s tracks analyzed." % (status,tracks_num)) 


In [ ]:
migration_speed_df.to_csv(output_folder + "migration_speeds_time_lag_%s.csv" % interval, index = False)

In [ ]:
print(interval)

# box plot effective path length

In [ ]:
plot_data_ = pd.DataFrame()

for filename in tracking_data_df["filename"].unique():
    tracking_data_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    for track_id in tracking_data_df_["TRACK_ID"].unique():
        single_track_df = tracking_data_df_[tracking_data_df_["TRACK_ID"]==track_id]
        if single_track_df["FRAME"].min() > 0:
            continue

        if len(plot_data_.index) > 3:
            plot_data_ = plot_data_.append(single_track_df)
        else:
            plot_data_ = single_track_df.copy()



In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

plt.rcParams.update({'font.size': 14})

plot_data = plot_data_[plot_data_["FRAME"]== single_time_point_evaluation].groupby(["condition", "filename"])["ORIGIN_X"].mean().reset_index()
#plot_data = single_time_point.copy()

sns.boxplot(data = plot_data, x = "condition", order = hue_order, y = "ORIGIN_X" )
sns.swarmplot(data = plot_data, x = "condition", order = hue_order, y = "ORIGIN_X" , color="black", size =10)
test_results = add_stat_annotation(ax, data=plot_data, y = "ORIGIN_X", x = "condition", order = hue_order, 
                                   #order=order,
                                   box_pairs=[("siScr", "siCdc42"), ("siScr", "siRac1"), ("siCdc42", "siRac1")],
                                   test='t-test_welch', text_format='star',
                                   loc='inside', verbose=2)

ax.set_ylabel("effective path length parallel to flow in $\mu$m")
ax.set_title("effective path length parallel to flow")
ax.set_ylim(-12.0,1.0)

plt.savefig(output_folder + "effective_path_length_parallel_to_flow_box_plot.png")
plt.savefig(output_folder + "effective_path_length_parallel_to_flow_box_plot.pdf")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

plot_data = plot_data_[plot_data_["FRAME"] == single_time_point_evaluation].groupby(["condition", "filename"])["ORIGIN_L"].mean().reset_index()


plt.rcParams.update({'font.size': 14})
sns.boxplot(data = plot_data, x = "condition", order = hue_order, y = "ORIGIN_L" )
sns.swarmplot(data = plot_data, x = "condition", order = hue_order, y = "ORIGIN_L" , color="black", size =10)
test_results = add_stat_annotation(ax, data=plot_data, y = "ORIGIN_L", x = "condition", order = hue_order, 
                                   #order=order,
                                   box_pairs=[("siScr", "siCdc42"), ("siScr", "siRac1"), ("siCdc42", "siRac1")],
                                   test='t-test_welch', text_format='star',
                                   loc='inside', verbose=2)

ax.set_ylabel("effective path length in $\mu$m")
ax.set_title("effective path length")
ax.set_ylim(0,24.0)

plt.savefig(output_folder + "effective_path_length_box_plot.png")
plt.savefig(output_folder + "effective_path_length_box_plot.pdf")